## Variables
These variables will be set by papermill

In [ ]:
x_start = 0
x_step = 0
y_start = 0
y_step = 0
z_start = 0
z_step = 0
gpu_idx = 0

weights_file = '../input/weights/membrane_weights.hdf5'
output_dir = '../output/untracked/run1'
file_prefix = f"bloss18_membrane_z{z_start}-{z_start+z_step}_y{y_start}-{y_start+y_step}_x{x_start}-{x_start+x_step}"


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_idx)

In [ ]:
from saber.cnn_tools import *
from saber.data_tools import *

import keras
from keras import backend as K
from keras.models import load_model

import sys
import time
import numpy as np
import json

from PIL import Image

from intern import array
import time


## Load specified chunk of data
Load the specified chunk of data for this job to process from the Boss.

In [ ]:
# Save a cutout to a numpy array in ZYX order:
em = array("bossdb://bloss/bloss18/image")
x_test = None
for _ in range(3):
    try:
        x_test = em[z_start:z_start+z_step, y_start:y_start+y_step, x_start:x_start+x_step]
        break
    except:
        print("retrying to download data....")
        time.sleep(5)
        
if x_test is None:
    raise Exception("Failed to download data after 3 tries!")

## Prepare the data
We're using some existing code that has all sorts of undocumented requirements. Let's just do what is recommended by reshaping and rescaling the data

In [ ]:
x_test = x_test[:, np.newaxis, :, :]
# Data must be [slices, chan, row, col] (i.e. [Z, chan, Y, X])
x_test = x_test.astype(np.float32)

In [ ]:
# Pixel values must be in [0,1]
if x_test.max() > 1.0:
    x_test /= 255.

## Load the Modal
Load the specified model and set the GPU in use

In [ ]:
width = 256
height = 256
tile_size = (width, height)
z_step_inf = 1

np.random.seed(9999)
K.common.set_image_dim_ordering('th')

with tf.device(f"/device:GPU:{gpu_idx}"):
    # Load model
    model = create_unet((1, int(width), int(height)));
    model.load_weights(weights_file);

## Classify images
Now we run images through the network. This will result in probability images from 0-255 indicating the probability that a pixel is a membrane (or synapse). You can then use this to threshold and classify pixels.

In [ ]:
tic = time.time()
y_hat = np.zeros(x_test.shape)
with tf.device(f"/device:GPU:{gpu_idx}"):
    for i in range(0, x_test.shape[0], z_step_inf):
        y_hat[i:i+z_step_inf, ...] = deploy_model(x_test[i:i+z_step_inf, ...], model)
        
    # Reshape for output
    y_hat = np.squeeze(np.transpose(y_hat, (3, 2, 0, 1)))
    y_hat = np.floor(y_hat*255)
    y_hat = y_hat.astype(np.uint8)
print('Total time to process entire volume: {}'.format(time.time() - tic))

## Save results
We save both a npy file and a stack of png images for later use and visualization

In [ ]:
# Save numpy array
np.save(os.path.join(output_dir, 'npy', f"{file_prefix}.npy"), y_hat)

# Save images
for idx in range(z_step):
    im = Image.fromarray(y_hat[:,:,idx])
    im.save(os.path.join(output_dir, 'png', f"bloss18_membrane_z{z_start+idx}_y{y_start}-{y_start+y_step}_x{x_start}-{x_start+x_step}.png"))
